In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout,Conv3D,Conv1D,Conv2D,Flatten,MaxPool2D
from sklearn.metrics import accuracy_score
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [6]:
import glob
files = glob.glob("../../datasets/prop-1.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [7]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0      prop-1        4    2    4    0    2    5     1   0   2  ...  1.000000   
1      prop-1        4   12    4    1   19   44    66   2  18  ...  0.000000   
2      prop-1        4   10    4    0   13   48    45   4  13  ...  1.000000   
3      prop-1        4    2    3    0    5    7     1   0   5  ...  1.000000   
4      prop-1        4    8    1    0    4   38     0   3   1  ...  0.500000   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
18466  prop-1      185   17    1    0   24   67    96   9  15  ...  0.150000   
18467  prop-1      185    9    1    0   25   49    14   7  18  ...  0.333333   
18468  prop-1      185    6    3    0   10   29     0   0  10  ...  0.800000   
18469  prop-1      185    5    5    0   28    9    10  26   2  ...  1.000000   
18470  prop-1      185   11    1    0   54   34     9  51   3  ...  0.666667   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0        0  0.941176  0.750000   1    1  10.500000       3  1.5000    0  
1        0  0.905172  0.350000   2    3  20.583333       8  2.0833    0  
2        0  0.918919  0.500000   2    3  19.100000       4  1.2000    0  
3        0  0.933333  0.750000   1    1  11.500000       4  2.0000    1  
4        0  0.000000  0.285714   0    0  30.625000       2  0.8750    0  
...    ...       ...       ...  ..  ...        ...     ...     ...  ...  
18466    0  0.000000  0.250000   0    0  41.529412       6  2.6471    1  
18467    0  0.000000  0.319444   0    0  29.888889       3  1.3333    0  
18468    0  0.818182  0.400000   0    0  45.500000       6  2.1667    0  
18469    0  0.962963  0.650000   1    1   4.400000       1  0.2000    0  
18470    1  0.000000  0.400000   0    0  25.545455       6  1.9091    0  

[18471 rows x 23 columns]

In [8]:
ant_master = ant_master.loc[ant_master['version'] == 121]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...  dam  moa  \
10152  prop-1      121    0    1    0    0    0     0   0   0  ...  0.0    0   
10153  prop-1      121    0    1    0    0    0     0   0   0  ...  0.0    0   
10154  prop-1      121    3    1    0    5   14     1   0   5  ...  0.5    0   
10155  prop-1      121    9    1    0   18   22    36   6  12  ...  0.0    0   
10156  prop-1      121    0    1    0    0    0     0   0   0  ...  0.0    0   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...  ...  ...   
13145  prop-1      121    2    1    0    5   12     1   0   5  ...  0.0    0   
13146  prop-1      121    1    1    0    0    5     0   0   0  ...  0.0    0   
13147  prop-1      121    1    1    0    0    5     0   0   0  ...  0.0    0   
13148  prop-1      121    1    1    0    0    5     0   0   0  ...  0.0    0   
13149  prop-1      121    1    1    0    0    5     0   0   0  ...  0.0    0   

       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
10152  0.0  0.000000   0    0   0.000000       0  0.0000    0  
10153  0.0  0.000000   0    0   0.000000       0  0.0000    0  
10154  0.0  0.500000   0    0  17.666667       1  0.3333    0  
10155  0.0  0.305556   0    0  11.888889       4  1.4444    1  
10156  0.0  0.000000   0    0   0.000000       0  0.0000    0  
...    ...       ...  ..  ...        ...     ...     ...  ...  
13145  0.0  0.625000   0    0  26.000000       4  2.0000    0  
13146  0.0  0.000000   0    0  12.000000       0  0.0000    0  
13147  0.0  0.000000   0    0  12.000000       0  0.0000    0  
13148  0.0  0.000000   0    0  12.000000       0  0.0000    0  
13149  0.0  0.000000   0    0  12.000000       0  0.0000    0  

[2998 rows x 23 columns]

In [9]:
train, test = train_test_split(ant_master, test_size=0.2)

In [10]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [11]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc       dam  \
12885    1    1    0   18    1     0  17   1    1  2.000000    1  0.000000   
11708    2    3    0    6    9     1   0   6    2  2.000000   27  0.000000   
12495    9    4    0   13   24    34   1  13    9  0.975000  105  0.800000   
11231    2    1    0    2    4     1   1   1    1  2.000000    9  0.000000   
12973    1    1    0    1    5     0   1   0    0  2.000000   17  0.000000   
...    ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...       ...   
11867   20    2    0    3   21   170   2   1   19  0.956938  127  0.636364   
11109    3    1    3    8    7     3   3   5    3  2.000000   37  0.000000   
11687    8    2    0   11   89    20   1  11    5  0.857143  694  0.333333   
12475    3    1    0    6   17     1   1   5    1  0.900000  137  0.200000   
10601    8    5    1   14   41    24   4  13    4  0.785714  174  1.000000   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  
12885    0  0.000000  1.000000   0    0   0.000000       1  1.0000  
11708    0  0.857143  0.666667   1    1  12.500000       2  1.0000  
12495    0  0.859649  1.000000   0    0  10.111111       2  1.1111  
11231    0  0.000000  0.500000   0    0   3.500000       1  0.5000  
12973    0  0.000000  0.000000   0    0  12.000000       0  0.0000  
...    ...       ...       ...  ..  ...        ...     ...     ...  
11867    0  0.419355  0.456140   1    5   4.800000       1  0.9000  
11109    0  0.000000  0.583333   0    0  11.333333       1  0.6667  
11687    1  0.800000  0.500000   2    6  85.000000      14  5.0000  
12475    0  0.000000  0.500000   0    0  43.000000       5  2.0000  
10601    0  0.932039  0.562500   1    2  20.500000       5  1.7500  

[2398 rows x 20 columns]

In [12]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc       dam  \
11201    5    6    0    3    9    10   1   2    5  1.250000   33  1.000000   
10359    3    2    0   16   41     0   1  15    1  0.666667  196  0.333333   
12762    4    1    0    4   31     0   0   4    2  0.833333  233  0.750000   
10736    9    3    0    3   18    16   0   3    8  0.375000   88  1.000000   
11164    1    1    0    3    1     0   1   2    1  2.000000    1  0.000000   
...    ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...       ...   
11849    9    1    0   18   80     2   1  18    5  0.812500  623  0.916667   
12610    6    2    3   11   16    15   3   8    4  2.000000   71  0.000000   
10352    2    1    0   11   14     1   0  11    2  2.000000   34  1.000000   
12948    5    2    0    5   46    10   1   5    2  2.000000  155  0.000000   
12940    2    3    0    7    9     1   0   7    1  2.000000   27  1.000000   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  
11201    0  0.962963  0.600000   1    1   5.400000       1  0.2000  
10359    0  0.900000  0.700000   1    1  63.333333      11  3.6667  
12762    1  0.000000  0.500000   0    0  56.250000       1  0.5000  
10736    0  0.829787  0.377778   2    7   8.666667       2  1.2222  
11164    0  0.000000  1.000000   0    0   0.000000       1  1.0000  
...    ...       ...       ...  ..  ...        ...     ...     ...  
11849    4  0.000000  0.562500   0    0  66.888889      22  4.0000  
12610    0  0.444444  0.472222   1    1  10.833333       2  1.1667  
10352    0  0.000000  0.666667   0    0  15.500000       1  0.5000  
12948    0  0.833333  0.500000   0    0  30.000000       2  0.8000  
12940    0  0.928571  0.750000   1    1  12.000000       2  1.0000  

[600 rows x 20 columns]

In [13]:
y_train = pd.DataFrame(train['bug'])
y_test = pd.DataFrame(test['bug'])

In [14]:
y_train

bug
12885    0
11708    0
12495    1
11231    0
12973    0
...    ...
11867    0
11109    0
11687    0
12475    0
10601    1

[2398 rows x 1 columns]

In [15]:
y_test

bug
11201    0
10359    3
12762    0
10736    0
11164    0
...    ...
11849    0
12610    0
10352    1
12948    0
12940    0

[600 rows x 1 columns]

In [16]:
#Getting the rows and columns size in our data
img_rows, img_cols = 1,20
# Preparing the data for the model.
X_train_matrix = x_train.values
X_test_matrix = x_test.values
Y_train_matrix = y_train.values
Y_test_matrix = y_test.values
# Preparing the data for the model.
Ytrainseries = y_train['bug']
Ytestseries = y_test['bug']

In [17]:
# Preparing the data for the model.
X_train1 = X_train_matrix.reshape(X_train_matrix.shape[0], img_rows, img_cols, 1)
X_test1 = X_test_matrix.reshape(X_test_matrix.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [18]:
def define_model():
    # Designing and initializing the model.
    #Building the model
    model = Sequential()
    #add model layers
    model.add(Conv1D(2, kernel_size=2, activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(Dense(1, activation='relu'))
    #compile model using mse as the loss function
    model.compile(loss='mse', optimizer = keras.optimizers.Adam(learning_rate=0.0005), metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()])
    return model

In [19]:
y = y_test['bug'].tolist()
y

[0,
 3,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 1,
 0,
 1,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 11,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,

In [20]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(30):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/cnn_propv121/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(X_train1, Y_train_matrix, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(X_test1, Y_test_matrix)
    total_scores.append(score) 
    predictions = model.predict(X_test1)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
24/24 [==============================] - 0s 2ms/step - loss: 6243.0244 - mse: 6243.0244 - mae: 29.1373 - root_mean_squared_error: 79.0128 - mean_squared_logarithmic_error: 6.8242
Epoch 2/100
24/24 [==============================] - 0s 1ms/step - loss: 5333.7080 - mse: 5333.7080 - mae: 26.2156 - root_mean_squared_error: 73.0322 - mean_squared_logarithmic_error: 6.3345
Epoch 3/100
24/24 [==============================] - 0s 1ms/step - loss: 4625.8828 - mse: 4625.8828 - mae: 23.5899 - root_mean_squared_error: 68.0138 - mean_squared_logarithmic_error: 5.8690
Epoch 4/100
24/24 [==============================] - 0s 959us/step - loss: 3989.1396 - mse: 3989.1396 - mae: 21.2069 - root_mean_squared_error: 63.1596 - mean_squared_logarithmic_error: 5.4098
Epoch 5/100
24/24 [==============================] - 0s 993us/step - loss: 3464.7866 - mse: 3464.7869 - mae: 19.1035 - root_mean_squared_error: 58.8624 - mean_squared_logarithmic_error: 4.9800
Epoch 6/100
24/24 [============

24/24 [==============================] - 0s 928us/step - loss: 65.3062 - mse: 65.3062 - mae: 0.6798 - root_mean_squared_error: 8.0812 - mean_squared_logarithmic_error: 0.2434
Epoch 87/100
24/24 [==============================] - 0s 942us/step - loss: 63.0133 - mse: 63.0133 - mae: 0.6710 - root_mean_squared_error: 7.9381 - mean_squared_logarithmic_error: 0.2414
Epoch 88/100
24/24 [==============================] - 0s 928us/step - loss: 61.4306 - mse: 61.4306 - mae: 0.6608 - root_mean_squared_error: 7.8378 - mean_squared_logarithmic_error: 0.2388
Epoch 89/100
24/24 [==============================] - 0s 920us/step - loss: 59.3187 - mse: 59.3187 - mae: 0.6496 - root_mean_squared_error: 7.7019 - mean_squared_logarithmic_error: 0.2362
Epoch 90/100
24/24 [==============================] - 0s 960us/step - loss: 57.9847 - mse: 57.9847 - mae: 0.6379 - root_mean_squared_error: 7.6148 - mean_squared_logarithmic_error: 0.2330
Epoch 91/100
24/24 [==============================] - 0s 940us/step - los

24/24 [==============================] - 0s 964us/step - loss: 2.1634 - mse: 2.1634 - mae: 0.2690 - root_mean_squared_error: 1.4709 - mean_squared_logarithmic_error: 0.1445
Epoch 71/100
24/24 [==============================] - 0s 976us/step - loss: 2.0848 - mse: 2.0848 - mae: 0.2683 - root_mean_squared_error: 1.4439 - mean_squared_logarithmic_error: 0.1444
Epoch 72/100
24/24 [==============================] - 0s 1ms/step - loss: 2.0119 - mse: 2.0119 - mae: 0.2677 - root_mean_squared_error: 1.4184 - mean_squared_logarithmic_error: 0.1443
Epoch 73/100
24/24 [==============================] - 0s 1ms/step - loss: 1.9193 - mse: 1.9193 - mae: 0.2668 - root_mean_squared_error: 1.3854 - mean_squared_logarithmic_error: 0.1442
Epoch 74/100
24/24 [==============================] - 0s 958us/step - loss: 1.8555 - mse: 1.8555 - mae: 0.2662 - root_mean_squared_error: 1.3622 - mean_squared_logarithmic_error: 0.1441
Epoch 75/100
24/24 [==============================] - 0s 2ms/step - loss: 1.7802 - mse:

24/24 [==============================] - 0s 965us/step - loss: 26.1985 - mse: 26.1985 - mae: 0.4469 - root_mean_squared_error: 5.1184 - mean_squared_logarithmic_error: 0.1854
Epoch 55/100
24/24 [==============================] - 0s 973us/step - loss: 24.4653 - mse: 24.4653 - mae: 0.4397 - root_mean_squared_error: 4.9462 - mean_squared_logarithmic_error: 0.1844
Epoch 56/100
24/24 [==============================] - 0s 954us/step - loss: 23.5584 - mse: 23.5584 - mae: 0.4353 - root_mean_squared_error: 4.8537 - mean_squared_logarithmic_error: 0.1838
Epoch 57/100
24/24 [==============================] - 0s 977us/step - loss: 22.5659 - mse: 22.5659 - mae: 0.4303 - root_mean_squared_error: 4.7504 - mean_squared_logarithmic_error: 0.1829
Epoch 58/100
24/24 [==============================] - 0s 968us/step - loss: 21.4602 - mse: 21.4602 - mae: 0.4250 - root_mean_squared_error: 4.6325 - mean_squared_logarithmic_error: 0.1820
Epoch 59/100
24/24 [==============================] - 0s 980us/step - los

24/24 [==============================] - 0s 917us/step - loss: 0.9449 - mse: 0.9449 - mae: 0.3103 - root_mean_squared_error: 0.9720 - mean_squared_logarithmic_error: 0.1585
Epoch 39/100
24/24 [==============================] - 0s 965us/step - loss: 0.9381 - mse: 0.9381 - mae: 0.3087 - root_mean_squared_error: 0.9686 - mean_squared_logarithmic_error: 0.1579
Epoch 40/100
24/24 [==============================] - 0s 961us/step - loss: 0.9322 - mse: 0.9322 - mae: 0.3070 - root_mean_squared_error: 0.9655 - mean_squared_logarithmic_error: 0.1572
Epoch 41/100
24/24 [==============================] - 0s 954us/step - loss: 0.9257 - mse: 0.9257 - mae: 0.3053 - root_mean_squared_error: 0.9621 - mean_squared_logarithmic_error: 0.1566
Epoch 42/100
24/24 [==============================] - 0s 969us/step - loss: 0.9204 - mse: 0.9204 - mae: 0.3036 - root_mean_squared_error: 0.9594 - mean_squared_logarithmic_error: 0.1559
Epoch 43/100
24/24 [==============================] - 0s 948us/step - loss: 0.9143 

...

24/24 [==============================] - 0s 837us/step - loss: 0.8015 - mse: 0.8015 - mae: 0.2452 - root_mean_squared_error: 0.8953 - mean_squared_logarithmic_error: 0.1376
Epoch 33/100
24/24 [==============================] - 0s 825us/step - loss: 0.8015 - mse: 0.8015 - mae: 0.2452 - root_mean_squared_error: 0.8953 - mean_squared_logarithmic_error: 0.1376
Epoch 34/100
24/24 [==============================] - 0s 824us/step - loss: 0.8015 - mse: 0.8015 - mae: 0.2452 - root_mean_squared_error: 0.8953 - mean_squared_logarithmic_error: 0.1376
Epoch 35/100
24/24 [==============================] - 0s 796us/step - loss: 0.8015 - mse: 0.8015 - mae: 0.2452 - root_mean_squared_error: 0.8953 - mean_squared_logarithmic_error: 0.1376
Epoch 36/100
24/24 [==============================] - 0s 786us/step - loss: 0.8015 - mse: 0.8015 - mae: 0.2452 - root_mean_squared_error: 0.8953 - mean_squared_logarithmic_error: 0.1376
Epoch 37/100
24/24 [==============================] - 0s 898us/step - loss: 0.8015 

24/24 [==============================] - 0s 876us/step - loss: 1.3130 - mse: 1.3130 - mae: 0.3632 - root_mean_squared_error: 1.1459 - mean_squared_logarithmic_error: 0.1979
Epoch 18/100
24/24 [==============================] - 0s 891us/step - loss: 1.2535 - mse: 1.2535 - mae: 0.3530 - root_mean_squared_error: 1.1196 - mean_squared_logarithmic_error: 0.1925
Epoch 19/100
24/24 [==============================] - 0s 1ms/step - loss: 1.1994 - mse: 1.1994 - mae: 0.3450 - root_mean_squared_error: 1.0952 - mean_squared_logarithmic_error: 0.1879
Epoch 20/100
24/24 [==============================] - 0s 1ms/step - loss: 1.1568 - mse: 1.1568 - mae: 0.3382 - root_mean_squared_error: 1.0756 - mean_squared_logarithmic_error: 0.1841
Epoch 21/100
24/24 [==============================] - 0s 1ms/step - loss: 1.1195 - mse: 1.1195 - mae: 0.3314 - root_mean_squared_error: 1.0581 - mean_squared_logarithmic_error: 0.1800
Epoch 22/100
24/24 [==============================] - 0s 1ms/step - loss: 1.0889 - mse: 1

24/24 [==============================] - 0s 765us/step - loss: 419.1537 - mse: 419.1537 - mae: 1.2961 - root_mean_squared_error: 20.4732 - mean_squared_logarithmic_error: 0.3126
Epoch 3/100
24/24 [==============================] - 0s 768us/step - loss: 394.3644 - mse: 394.3644 - mae: 1.2518 - root_mean_squared_error: 19.8586 - mean_squared_logarithmic_error: 0.3068
Epoch 4/100
24/24 [==============================] - 0s 792us/step - loss: 373.0075 - mse: 373.0075 - mae: 1.1984 - root_mean_squared_error: 19.3134 - mean_squared_logarithmic_error: 0.2959
Epoch 5/100
24/24 [==============================] - 0s 743us/step - loss: 350.7633 - mse: 350.7633 - mae: 1.1503 - root_mean_squared_error: 18.7287 - mean_squared_logarithmic_error: 0.2886
Epoch 6/100
24/24 [==============================] - 0s 789us/step - loss: 334.9119 - mse: 334.9119 - mae: 1.1203 - root_mean_squared_error: 18.3006 - mean_squared_logarithmic_error: 0.2826
Epoch 7/100
24/24 [==============================] - 0s 836us/

24/24 [==============================] - 0s 840us/step - loss: 5.9862 - mse: 5.9862 - mae: 0.3002 - root_mean_squared_error: 2.4467 - mean_squared_logarithmic_error: 0.1519
Epoch 89/100
24/24 [==============================] - 0s 1ms/step - loss: 5.8115 - mse: 5.8115 - mae: 0.2989 - root_mean_squared_error: 2.4107 - mean_squared_logarithmic_error: 0.1516
Epoch 90/100
24/24 [==============================] - 0s 871us/step - loss: 5.6614 - mse: 5.6614 - mae: 0.2978 - root_mean_squared_error: 2.3794 - mean_squared_logarithmic_error: 0.1513
Epoch 91/100
24/24 [==============================] - 0s 788us/step - loss: 5.5678 - mse: 5.5678 - mae: 0.2969 - root_mean_squared_error: 2.3596 - mean_squared_logarithmic_error: 0.1510
Epoch 92/100
24/24 [==============================] - 0s 799us/step - loss: 5.3872 - mse: 5.3872 - mae: 0.2957 - root_mean_squared_error: 2.3210 - mean_squared_logarithmic_error: 0.1508
Epoch 93/100
24/24 [==============================] - 0s 829us/step - loss: 5.3430 - 

24/24 [==============================] - 0s 846us/step - loss: 87.0936 - mse: 87.0936 - mae: 0.6184 - root_mean_squared_error: 9.3324 - mean_squared_logarithmic_error: 0.2027
Epoch 72/100
24/24 [==============================] - 0s 789us/step - loss: 83.7359 - mse: 83.7359 - mae: 0.6094 - root_mean_squared_error: 9.1507 - mean_squared_logarithmic_error: 0.2016
Epoch 73/100
24/24 [==============================] - 0s 934us/step - loss: 80.1745 - mse: 80.1745 - mae: 0.5992 - root_mean_squared_error: 8.9540 - mean_squared_logarithmic_error: 0.2004
Epoch 74/100
24/24 [==============================] - 0s 1ms/step - loss: 76.9577 - mse: 76.9577 - mae: 0.5898 - root_mean_squared_error: 8.7726 - mean_squared_logarithmic_error: 0.1990
Epoch 75/100
24/24 [==============================] - 0s 840us/step - loss: 73.6555 - mse: 73.6555 - mae: 0.5814 - root_mean_squared_error: 8.5823 - mean_squared_logarithmic_error: 0.1982
Epoch 76/100
24/24 [==============================] - 0s 791us/step - loss:

In [21]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/cnn_propv121/'

In [22]:
total_training_time

[3.3213393688201904,
 3.159503936767578,
 3.141448497772217,
 3.1046907901763916,
 3.2735540866851807,
 3.2478675842285156,
 3.323478937149048,
 2.9812495708465576,
 2.7816877365112305,
 2.785637140274048,
 2.7439091205596924,
 2.7929553985595703,
 2.8345065116882324,
 3.060816764831543,
 2.860487222671509,
 2.8336544036865234,
 2.7929465770721436,
 2.798851728439331,
 2.780115842819214,
 2.7279016971588135,
 2.824679374694824,
 3.0989201068878174,
 3.2834765911102295,
 3.3234333992004395,
 2.7887985706329346,
 2.863333225250244,
 2.843958616256714,
 2.918916702270508,
 2.84342622756958,
 2.9099173545837402]

In [23]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 2.968182102839152


In [24]:
total_training_loss_history

[{'loss': [6243.0244140625,
   5333.7080078125,
   4625.8828125,
   3989.1396484375,
   3464.78662109375,
   3027.171142578125,
   2672.80224609375,
   2344.509521484375,
   2088.3662109375,
   1874.5596923828125,
   1686.604248046875,
   1517.738037109375,
   1382.1514892578125,
   1269.089599609375,
   1164.034423828125,
   1074.4315185546875,
   999.7933959960938,
   931.2881469726562,
   880.514404296875,
   818.4754638671875,
   775.3562622070312,
   731.099609375,
   693.1057739257812,
   656.0399780273438,
   626.5223999023438,
   593.468017578125,
   568.2012329101562,
   542.318603515625,
   515.8850708007812,
   494.9692077636719,
   471.4061584472656,
   452.463623046875,
   432.4399719238281,
   414.9897766113281,
   395.49920654296875,
   381.3135070800781,
   364.56744384765625,
   349.1498718261719,
   334.45306396484375,
   324.8230895996094,
   311.8551025390625,
   297.2133483886719,
   285.7242736816406,
   275.4801330566406,
   264.3119812011719,
   255.803482055664

In [25]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(30):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 7.852144026756287
avg mae of training: 0.30076695879300436


In [26]:
total_scores

[[35.9605827331543,
  35.9605827331543,
  0.740519106388092,
  5.996714115142822,
  0.2382679432630539],
 [1.0433332920074463,
  1.0433332920074463,
  0.2666666805744171,
  1.0214369297027588,
  0.15116705000400543],
 [5.7666497230529785,
  5.7666497230529785,
  0.3740626573562622,
  2.4013850688934326,
  0.1888115108013153],
 [1.0037124156951904,
  1.0037124156951904,
  0.26639601588249207,
  1.0018545389175415,
  0.14457650482654572],
 [1.04279625415802,
  1.04279625415802,
  0.26697519421577454,
  1.0211739540100098,
  0.15086255967617035],
 [1.0433332920074463,
  1.0433332920074463,
  0.2666666805744171,
  1.0214369297027588,
  0.15116705000400543],
 [3.395777940750122,
  3.395777940750122,
  0.3292824923992157,
  1.8427636623382568,
  0.17340116202831268],
 [0.6970680356025696,
  0.6970680356025696,
  0.26028451323509216,
  0.8349059820175171,
  0.1255997121334076],
 [0.8664625287055969,
  0.8664625287055969,
  0.27734267711639404,
  0.9308397173881531,
  0.14124667644500732],
 [1

In [4]:
a = [[35.9605827331543,
  35.9605827331543,
  0.740519106388092,
  5.996714115142822,
  0.2382679432630539],
 [1.0433332920074463,
  1.0433332920074463,
  0.2666666805744171,
  1.0214369297027588,
  0.15116705000400543],
 [5.7666497230529785,
  5.7666497230529785,
  0.3740626573562622,
  2.4013850688934326,
  0.1888115108013153],
 [1.0037124156951904,
  1.0037124156951904,
  0.26639601588249207,
  1.0018545389175415,
  0.14457650482654572],
 [1.04279625415802,
  1.04279625415802,
  0.26697519421577454,
  1.0211739540100098,
  0.15086255967617035],
 [1.0433332920074463,
  1.0433332920074463,
  0.2666666805744171,
  1.0214369297027588,
  0.15116705000400543],
 [3.395777940750122,
  3.395777940750122,
  0.3292824923992157,
  1.8427636623382568,
  0.17340116202831268],
 [0.6970680356025696,
  0.6970680356025696,
  0.26028451323509216,
  0.8349059820175171,
  0.1255997121334076],
 [0.8664625287055969,
  0.8664625287055969,
  0.27734267711639404,
  0.9308397173881531,
  0.14124667644500732],
 [1.0433332920074463,
  1.0433332920074463,
  0.26667726039886475,
  1.0214369297027588,
  0.1511671096086502],
 [1.0433332920074463,
  1.0433332920074463,
  0.26667124032974243,
  1.0214369297027588,
  0.15116705000400543],
 [1.0433332920074463,
  1.0433332920074463,
  0.2666666805744171,
  1.0214369297027588,
  0.15116705000400543],
 [1.0433332920074463,
  1.0433332920074463,
  0.2666666805744171,
  1.0214369297027588,
  0.15116705000400543],
 [1.0433332920074463,
  1.0433332920074463,
  0.2666666805744171,
  1.0214369297027588,
  0.15116705000400543],
 [1.0438690185546875,
  1.0438690185546875,
  0.2767315208911896,
  1.021699070930481,
  0.1520967036485672],
 [1.030250906944275,
  1.030250906944275,
  0.2655986249446869,
  1.0150127410888672,
  0.14768457412719727],
 [1.1904345750808716,
  1.1904345750808716,
  0.2935712933540344,
  1.0910704135894775,
  0.1667003482580185],
 [0.9865236282348633,
  0.9865236282348633,
  0.261208176612854,
  0.9932389855384827,
  0.1418723613023758],
 [65.62653350830078,
  65.62653350830078,
  0.6983651518821716,
  8.101020812988281,
  0.23113438487052917],
 [1.0433332920074463,
  1.0433332920074463,
  0.2666666805744171,
  1.0214369297027588,
  0.15116705000400543],
 [1.0433332920074463,
  1.0433332920074463,
  0.2666666805744171,
  1.0214369297027588,
  0.15116705000400543],
 [23.35614776611328,
  23.35614776611328,
  0.456954687833786,
  4.832819938659668,
  0.17547707259655],
 [1.0433332920074463,
  1.0433332920074463,
  0.2666666805744171,
  1.0214369297027588,
  0.15116705000400543],
 [22.125001907348633,
  22.125001907348633,
  0.45245370268821716,
  4.70372200012207,
  0.1776657998561859],
 [1.0433332920074463,
  1.0433332920074463,
  0.2666666805744171,
  1.0214369297027588,
  0.15116705000400543],
 [1.0388500690460205,
  1.0388500690460205,
  0.27959463000297546,
  1.019239902496338,
  0.15039178729057312],
 [1.0433193445205688,
  1.0433193445205688,
  0.2666596472263336,
  1.0214300155639648,
  0.15115737915039062],
 [1.0274890661239624,
  1.0274890661239624,
  0.2658859193325043,
  1.0136513710021973,
  0.14729130268096924],
 [3.8304615020751953,
  3.8304615020751953,
  0.34038102626800537,
  1.9571565389633179,
  0.17812715470790863],
 [34.10659408569336,
  34.10659408569336,
  0.5541439652442932,
  5.840085029602051,
  0.21338854730129242]]

In [5]:
test_mae = []
for i in range(10):
    test_mae.append(a[i][2])
sum(test_mae)/10

0.3314873278141022

In [3]:
test_mae

[0.740519106388092,
 0.2666666805744171,
 0.3740626573562622,
 0.26639601588249207,
 0.26697519421577454,
 0.2666666805744171,
 0.3292824923992157,
 0.26028451323509216,
 0.27734267711639404,
 0.26667726039886475,
 0.26667124032974243,
 0.2666666805744171,
 0.2666666805744171,
 0.2666666805744171,
 0.2767315208911896,
 0.2655986249446869,
 0.2935712933540344,
 0.261208176612854,
 0.6983651518821716,
 0.2666666805744171,
 0.2666666805744171,
 0.456954687833786,
 0.2666666805744171,
 0.45245370268821716,
 0.2666666805744171,
 0.27959463000297546,
 0.2666596472263336,
 0.2658859193325043,
 0.34038102626800537,
 0.5541439652442932]

In [30]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(30):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 7.220506374041239
avg mae: 0.3286586542924245
avg are: 0.15169580529133478


In [31]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 0.8018686473369598
median testing mse: 1.0433332920074463


In [32]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.25089358538389206
median testing mae: 0.2666689604520798


In [33]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.13833333551883698


In [34]:
test_ARE

[0.17646465,
 0.13833334,
 0.13833334,
 0.13599999,
 0.13833334,
 0.13833334,
 0.13833334,
 0.13555844,
 0.13534091,
 0.13833334,
 0.13833334,
 0.13833334,
 0.13833334,
 0.13833334,
 0.13833334,
 0.13833334,
 0.13833334,
 0.13733333,
 0.13601011,
 0.13833334,
 0.13833334,
 0.32833335,
 0.13833334,
 0.32333332,
 0.13833334,
 0.13833334,
 0.13833334,
 0.1375,
 0.13833334,
 0.13833334]

In [6]:
sum([0.17646465,
 0.13833334,
 0.13833334,
 0.13599999,
 0.13833334,
 0.13833334,
 0.13833334,
 0.13555844,
 0.13534091,
 0.13833334,
 0.13833334,
 0.13833334,
 0.13833334,
 0.13833334,
 0.13833334,
 0.13833334,
 0.13833334,
 0.13733333,
 0.13601011,
 0.13833334,
 0.13833334,
 0.32833335,
 0.13833334,
 0.32333332,
 0.13833334,
 0.13833334,
 0.13833334,
 0.1375,
 0.13833334,
 0.13833334][:10])/10

0.141336403